In [1]:

!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 535 (delta 174), reused 130 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (535/535), 172.47 KiB | 1003.00 KiB/s, done.
Resolving deltas: 100% (276/276), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.2 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px
from cuml import train_test_split as cuml_train_test_split  # Updated import for train_test_split
from cuml.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
# from cuml.imblearn.over_sampling import SMOTE as cuSMOTE  # Original import causing the error
# from imblearn.over_sampling import SMOTE  # Updated import using imblearn directly


In [53]:
data = pd.read_csv('MHD.csv')
stress_per_country = data[data['Growing_Stress'] == 'Yes'].groupby('Country').size().sort_values(ascending=False)
country_stress_counts = data[data['Growing_Stress'] == 'Yes'].groupby('Country').size().reset_index(name='stress_count')

In [54]:
# Drop irrelevant columns
data = data.drop(columns=['Timestamp', 'Country', 'Occupation'])

# Convert boolean columns to integers
data = data.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

# Separate the target before encoding and imputation
target = data['treatment']
features = data.drop(columns=['treatment'])





In [55]:
from cuml.preprocessing import StandardScaler  # Import cuStandardScaler

In [56]:
# Encode categorical features using one-hot encoding
features_encoded = pd.get_dummies(features)
features_encoded = features_encoded.map(lambda x: int(x) if isinstance(x, bool) else x)

# Handle missing values with the most frequent strategy
imputer = SimpleImputer(strategy='most_frequent')
features_imputed = pd.DataFrame(imputer.fit_transform(features_encoded), columns=features_encoded.columns)

label_encoder = LabelEncoder()  # Create a LabelEncoder object
target_encoded = label_encoder.fit_transform(target)  # Fit and transform the target variable
# Split into training and testing sets
X_train, X_test, y_train, y_test = cuml_train_test_split(features_imputed, target_encoded, test_size=0.3, random_state=42)
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [58]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
#agar tonumpy error show kare to ek bar upar wala uncomment kar


# Logistic Regression
logistic_model = LogisticRegression(max_iter=200, random_state=42)
logistic_param_grid = {'C': [0.1, 1, 10], 'penalty': ['l2']}
logistic_grid = GridSearchCV(logistic_model, logistic_param_grid, cv=5, scoring='accuracy')
logistic_grid.fit(X_train, y_train)
logistic_best = logistic_grid.best_estimator_


In [59]:
decision_tree_model = DecisionTreeClassifier(random_state=42)
tree_param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
tree_grid = GridSearchCV(decision_tree_model, tree_param_grid, cv=5, scoring='accuracy')
tree_grid.fit(X_train, y_train)
tree_best = tree_grid.best_estimator_

In [60]:
from sklearn.model_selection import RandomizedSearchCV
random_forest_model = cuRF(random_state=42)

# Define a smaller parameter grid or use RandomizedSearchCV
rf_param_grid = {
    'n_estimators': [50, 100],  # fewer values to speed up
    'max_depth': [10, 20, None],  # fewer values
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Use RandomizedSearchCV with n_iter to limit combinations tested
rf_random_search = RandomizedSearchCV(
    random_forest_model, rf_param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42
)
rf_random_search.fit(X_train, y_train)
rf_best = rf_random_search.best_estimator_

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results 

In [61]:
# Fit the best models on the training data
logistic_best.fit(X_train, y_train)
tree_best.fit(X_train, y_train)
rf_best.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


RandomForestClassifier()

In [63]:
logistic_predictions = logistic_best.predict(X_test)
decision_tree_predictions = tree_best.predict(X_test)
random_forest_predictions = rf_best.predict(X_test)

logistic_accuracy = accuracy_score(y_test, logistic_predictions)
decision_tree_accuracy = accuracy_score(y_test, decision_tree_predictions)
random_forest_accuracy = accuracy_score(y_test, random_forest_predictions)

print("Logistic Regression Accuracy:", logistic_accuracy)
print("Decision Tree Accuracy:", decision_tree_accuracy)
print("Random Forest Accuracy:", random_forest_accuracy)

print("\nLogistic Regression Classification Report:\n", classification_report(y_test, logistic_predictions))
print("Decision Tree Classification Report:\n", classification_report(y_test, decision_tree_predictions))
print("Random Forest Classification Report:\n", classification_report(y_test, random_forest_predictions))

Logistic Regression Accuracy: 0.7039300203323364
Decision Tree Accuracy: 0.7221266031265259
Random Forest Accuracy: 0.7226738333702087

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.68      0.69     43315
           1       0.70      0.73      0.71     44394

    accuracy                           0.70     87709
   macro avg       0.70      0.70      0.70     87709
weighted avg       0.70      0.70      0.70     87709

Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.66      0.70     43315
           1       0.70      0.78      0.74     44394

    accuracy                           0.72     87709
   macro avg       0.73      0.72      0.72     87709
weighted avg       0.72      0.72      0.72     87709

Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.68  

In [64]:
from cuml.svm import SVC as cumlSVC
from sklearn.model_selection import GridSearchCV
from cuml.metrics import accuracy_score
from sklearn.metrics import classification_report

# Initialize the GPU-accelerated SVM model
svm_model = cumlSVC(kernel='linear', probability=True)

# Train the SVM model on GPU
svm_model.fit(X_train, y_train)

# Make predictions with the SVM model
svm_predictions = svm_model.predict(X_test)

# Evaluate the SVM model
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_report = classification_report(y_test, svm_predictions)

# Print the accuracy and classification report for the SVM model
print("SVM Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", svm_report)


RuntimeError: The work in this thread was cancelled.
Obtained 63 stack frames
#1 in /usr/local/lib/python3.10/dist-packages/pylibraft/common/handle.cpython-310-x86_64-linux-gnu.so: void raft::interruptible::synchronize_impl<cudaError (*)(CUstream_st*), rmm::cuda_stream_view>(cudaError (*)(CUstream_st*), rmm::cuda_stream_view) +0x2e6 [0x79ba62af1be6]
#2 in /usr/local/lib/python3.10/dist-packages/pylibraft/common/handle.cpython-310-x86_64-linux-gnu.so(+0x43a1a) [0x79ba62adaa1a]
#3 in /usr/local/lib/python3.10/dist-packages/cuml/svm/svc.cpython-310-x86_64-linux-gnu.so(+0x43b90) [0x79ba4fc07b90]
#4 in /usr/bin/python3: _PyEval_EvalFrameDefault +0x285e [0x5a5cad9e6a6e]
#5 in /usr/bin/python3(+0x16b3ce) [0x5a5cada0a3ce]
#6 in /usr/bin/python3: _PyEval_EvalFrameDefault +0x285e [0x5a5cad9e6a6e]
#7 in /usr/bin/python3: _PyFunction_Vectorcall +0x7c [0x5a5cad9fc42c]
#8 in /usr/bin/python3: PyObject_Call +0x122 [0x5a5cada0af22]
#9 in /usr/bin/python3: _PyEval_EvalFrameDefault +0x285e [0x5a5cad9e6a

In [65]:

# Encode target labels as 0 and 1
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.3, 0.7, 1.0]
}

# Randomized search with cross-validation
xgb_random_search = RandomizedSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42,tree_method='gpu_hist'),
    param_distributions=param_grid,
    n_iter=20,  # number of parameter settings sampled
    scoring='accuracy',
    cv=5,
    random_state=42,
    n_jobs=-1
)

xgb_random_search.fit(X_train, y_train)
xgb_best = xgb_random_search.best_estimator_

# Evaluate the tuned model
xgb_predictions = xgb_best.predict(X_test)
xgb_accuracy = accuracy_score(y_test_encoded, xgb_predictions)
xgb_report = classification_report(y_test_encoded, xgb_predictions)

print("Tuned XGBoost Accuracy:", xgb_accuracy)
print("Tuned XGBoost Classification Report:\n", xgb_report)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:56:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:56:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Tuned XGBoost Accuracy: 0.7244410514831543
Tuned XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.67      0.71     43315
           1       0.71      0.78      0.74     44394

    accuracy                           0.72     87709
   macro avg       0.73      0.72      0.72     87709
weighted avg       0.73      0.72      0.72     87709



Logistic Regression Accuracy: 0.7126666666666667


In [13]:
!python rapidsai-csp-utils/colab/pip-install.py --stable

Installing RAPIDS Stable 24.10.*
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com

        ***********************************************************************
        The pip install of RAPIDS is complete.
        
        Please do not run any further installation from the conda based installation methods, as they may cause issues!
        
        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files. 
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        ***********************************************************************
        


In [9]:
!pip install cuml imbalanced-learn

  Using cached cuml-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cuml
  Running setup.py clean for cuml
Failed to build cuml
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cuml)
